### Proyecto sobre el analisis exploratorio de datos, en los jugadores. 
*Usando la libreria Polars*

### Problema Planteado a resolver

**Planteamiento:**
Un club de fútbol desea identificar los factores que influyen en el desempeño de sus jugadores. El club considera que variables como el número de prácticas, horas de entrenamiento, asistencia al gimnasio, hábitos de fumar, nutrición y horas en casa podrían estar relacionadas con el rendimiento total del jugador.

**Preguntas a responder con el análisis exploratorio:**

*Distribución general de los jugadores*

¿Cómo se distribuyen los futbolistas por liga (Primera, Segunda, Tercera)?

¿Cuál es la distribución de edades y alturas de los jugadores?

**Factores de entrenamiento y rendimiento**

¿Los jugadores que entrenan más horas tienen un mayor puntaje en Total?

¿La asistencia al gimnasio influye en el rendimiento promedio?

¿El hábito de fumar tiene relación con el rendimiento (Total)?

**Aspectos económicos**

¿Cuál es el precio promedio de un jugador en cada liga?

¿Existe una correlación entre el rendimiento (Total) y el precio de mercado (Precio)?

**Hábitos de vida y desempeño**

¿Los jugadores que cuentan con nutriólogo obtienen mejores resultados en Total?

¿El número de horas en casa influye en el rendimiento?

In [422]:
#Importar la librerias 
import polars as pl
import matplotlib as plt
from datetime import datetime
import pandas as pandas

##### Lectura de la informacion

In [423]:
df = pl.read_csv('futbolista.csv')
df.head()

Id,Nombre,Liga,Practicas,Horas_Entrenando,Altura,Precio,Juegos,Gimnasio,Fuma,Total,Nacimiento,Nutriologo,Horas_Casa
i64,str,str,i64,i64,f64,str,i64,str,str,f64,str,str,i64
1,"""Casimiro Vargas""","""Primera""",4,7,1.687905,"""1017076.54""",6,"""Si""","""No""",93.38,"""22/04/1998""","""No""",1
2,"""Máximo Santillas""","""Tercera""",4,5,1.802727,"""91239.66""",6,"""Si""","""Si""",76.64,"""17/10/1998""","""Si""",2
3,"""Wilma Vera""","""Primera""",2,2,1.834843,"""1764449.46""",2,"""Si""","""No""",82.31,"""27/11/1998""","""No""",5
4,"""Douglas Vadillo""","""Segunda""",3,7,1.690813,"""464105.59""",4,"""No""","""No""",78.72,"""13/06/1998""","""Si""",1
5,"""Maria Blanchet""","""Segunda""",1,2,1.678295,"""461403.91""",6,"""Si""","""Si""",81.15,"""09/10/1998""","""No""",4


In [424]:

df.shape # ver la cantidad de columna
#Ver el tipo de datos de cada columna
df.schema

Schema([('Id', Int64),
        ('Nombre', String),
        ('Liga', String),
        ('Practicas', Int64),
        ('Horas_Entrenando', Int64),
        ('Altura', Float64),
        ('Precio ', String),
        ('Juegos', Int64),
        ('Gimnasio', String),
        ('Fuma', String),
        (' Total ', Float64),
        ('Nacimiento', String),
        ('Nutriologo', String),
        ('Horas_Casa', Int64)])

In [425]:
#Limpiar encabezados con espacios
df = df.rename({c: c.strip() for c in df.columns})
#Convertir la columna nacimiento a tipo fecha
df = df.with_columns(
    pl.col('Nacimiento').cast(pl.Utf8).str.strip_chars().str.replace_all(r'[-.]', '/')
    .str.to_date(format='%d/%m/%Y', strict=False)
    .alias('Nacimiento')
)
df

Id,Nombre,Liga,Practicas,Horas_Entrenando,Altura,Precio,Juegos,Gimnasio,Fuma,Total,Nacimiento,Nutriologo,Horas_Casa
i64,str,str,i64,i64,f64,str,i64,str,str,f64,date,str,i64
1,"""Casimiro Vargas""","""Primera""",4,7,1.687905,"""1017076.54""",6,"""Si""","""No""",93.38,1998-04-22,"""No""",1
2,"""Máximo Santillas""","""Tercera""",4,5,1.802727,"""91239.66""",6,"""Si""","""Si""",76.64,1998-10-17,"""Si""",2
3,"""Wilma Vera""","""Primera""",2,2,1.834843,"""1764449.46""",2,"""Si""","""No""",82.31,1998-11-27,"""No""",5
4,"""Douglas Vadillo""","""Segunda""",3,7,1.690813,"""464105.59""",4,"""No""","""No""",78.72,1998-06-13,"""Si""",1
5,"""Maria Blanchet""","""Segunda""",1,2,1.678295,"""461403.91""",6,"""Si""","""Si""",81.15,1998-10-09,"""No""",4
…,…,…,…,…,…,…,…,…,…,…,…,…,…
396,"""Favio Olvera""","""Segunda""",8,10,1.64861,"""437889.62""",6,"""No""","""No""",105.01,1998-07-19,"""Si""",0
397,"""Erika Villalpando""","""Primera""",3,6,1.734986,"""1465194.27""",10,"""No""","""No""",86.86,1998-02-28,"""Si""",6
398,"""Alan Mansilla""","""Segunda""",2,3,1.732246,"""503936.10""",6,"""Si""","""No""",70.21,1998-12-06,"""Si""",4


In [426]:
#Dispribucion de futbolista por ligas
futbolista_Liga = df.group_by('Liga').agg(pl.count()).sort('count', descending=True)
futbolista_Liga

C:\Users\monti.UNITY12\AppData\Local\Temp\ipykernel_10344\712404724.py:2: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  futbolista_Liga = df.group_by('Liga').agg(pl.count()).sort('count', descending=True)


Liga,count
str,u32
"""Primera""",184
"""Segunda""",128
"""Tercera""",58
"""Ninguna""",30


In [427]:
# Crear columna Edad
from datetime import date

#Distribuir las edades de los jugadores
#Calcular la edad apartir de la fecha actual y la de nacimiento
hoy = date.today()
df = df.with_columns(
    hoy.year - pl.col('Nacimiento').dt.year().alias('Edad')
)
df.head()

Id,Nombre,Liga,Practicas,Horas_Entrenando,Altura,Precio,Juegos,Gimnasio,Fuma,Total,Nacimiento,Nutriologo,Horas_Casa,literal
i64,str,str,i64,i64,f64,str,i64,str,str,f64,date,str,i64,i32
1,"""Casimiro Vargas""","""Primera""",4,7,1.687905,"""1017076.54""",6,"""Si""","""No""",93.38,1998-04-22,"""No""",1,27
2,"""Máximo Santillas""","""Tercera""",4,5,1.802727,"""91239.66""",6,"""Si""","""Si""",76.64,1998-10-17,"""Si""",2,27
3,"""Wilma Vera""","""Primera""",2,2,1.834843,"""1764449.46""",2,"""Si""","""No""",82.31,1998-11-27,"""No""",5,27
4,"""Douglas Vadillo""","""Segunda""",3,7,1.690813,"""464105.59""",4,"""No""","""No""",78.72,1998-06-13,"""Si""",1,27
5,"""Maria Blanchet""","""Segunda""",1,2,1.678295,"""461403.91""",6,"""Si""","""Si""",81.15,1998-10-09,"""No""",4,27


In [428]:
#Jugadores que entrenan mas horas y tienen mayor puntaje total
relacion_horas_puntaje = (df.group_by('Horas_Entrenando')
                          .agg(pl.col('Total').mean().alias('Puntaje Promedio')).sort('Horas_Entrenando', descending=True)
)
relacion_horas_puntaje

Horas_Entrenando,Puntaje Promedio
i64,f64
24,152.92
23,196.535
22,174.035
21,150.8
20,180.6925
…,…
5,81.616667
4,80.645294
3,71.894603


In [429]:
#Promedio de rendimiento según gimnasio
promedio_rendimiento = df.group_by('Gimnasio').agg(pl.col('Total')
                                                   .mean().round()
                                                   .alias('Promedio rendimiento')).sort('Promedio rendimiento', descending=True)
promedio_rendimiento

Gimnasio,Promedio rendimiento
str,f64
"""Si""",96.0
"""No""",89.0


In [430]:
#Relación entre fumar y rendimiento
#Crear una tabla de contingencia usando una tabla dinamica para mostra la informacion
tabla_contingencia = df.pivot(values='Total', index='Fuma', columns='Gimnasio', aggregate_function='mean').with_columns(
    [pl.col(pl.Float64).round(2)]  # solo redondea columnas tipo float
)
tabla_contingencia


C:\Users\monti.UNITY12\AppData\Local\Temp\ipykernel_10344\3236656341.py:3: DeprecationWarning: the argument `columns` for `DataFrame.pivot` is deprecated. It was renamed to `on` in version 1.0.0.
  tabla_contingencia = df.pivot(values='Total', index='Fuma', columns='Gimnasio', aggregate_function='mean').with_columns(


Fuma,Si,No
str,f64,f64
"""No""",101.35,93.96
"""Si""",92.55,84.45


In [431]:
#¿Cuál es el precio promedio de un jugador en cada liga?
#convertir la columna precio que esta detectada como formato de cadena, convirtiendola a formato numerico

df = df.with_columns(
    pl.col("Precio")
    .str.replace_all(r"\.", "")       # Elimina puntos (miles)
    .str.replace(",", ".")            # Convierte coma decimal a punto
    .map_elements(lambda x: float(x) if x.replace(".", "", 1).isdigit() else None)
    .alias("Precio")
)

#Agrupar
promedio_liga = df.group_by('Liga').agg(pl.col('Precio').mean().round(2).alias('Precio Promedio'))
promedio_liga

C:\Users\monti.UNITY12\AppData\Local\Temp\ipykernel_10344\1312110724.py:4: MapWithoutReturnDtypeWarning: 'return_dtype' of function python_udf must be set

A later expression might fail because the output type is not known. Set return_dtype=pl.self_dtype() if the type is unchanged, or set the proper output data type.
  df = df.with_columns(


Liga,Precio Promedio
str,f64
"""Tercera""",7.9507e6
"""Segunda""",4.7274e7
"""Ninguna""",null
"""Primera""",1.0143e8


ModuleNotFoundError: No module named 'pyarrow'